In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import pickle

data_file = open("data_input_lucrezia.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("pars_lucrezia.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface()

In [3]:
pyro.enable_validation()

In [16]:
param["init_probs"] = 0.55
param["latent_type"] = "C"
param["likelihood_atac"] = "G"
param["lambda"] = 0.3
param["K"] = 3
param["Temperature"] = 10

In [17]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)

In [18]:
ll = interface.run(steps = 100, param_optimizer = {"lr":0.01})



Running LatentCategorical on 3768 cells with 20 segments for 100 steps and 2 modalities
tensor([[0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]])


Bar desc:   0%|          | 0/100 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'detach'

In [8]:

import numpy as np

lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.


In [9]:
lr

{'mixture_weights_rna': array([0.5, 0.5], dtype=float32),
 'mixture_weights_atac': array([0.61508876, 0.38491124], dtype=float32),
 'NB_size_rna': array([149.99998, 149.99998, 149.99998, 149.99998, 149.99998, 149.99998,
        149.99998, 149.99998, 149.99998, 149.99998], dtype=float32),
 'segment_factor_rna': array([14.138505 ,  4.7261987,  2.2322578,  4.8723235,  3.2121549,
        12.320882 , 12.533794 ,  1.9288603,  3.979431 ,  2.9039323],
       dtype=float32),
 'NB_size_atac': array([61.962257, 61.962257, 61.962257, 61.962257, 61.962257, 61.962257,
        61.962257, 61.962257, 61.962257, 61.962257], dtype=float32),
 'segment_factor_atac': array([14.605779, 11.870394,  7.349555,  7.300733,  7.186394, 17.984558,
        20.219145,  8.637547,  8.063735, 10.507065], dtype=float32),
 'CNV_probabilities': array([[[0.02831742, 0.02831742, 0.02831742, 0.02831742, 0.02831742,
          0.858413  ],
         [0.02831742, 0.02831742, 0.02831742, 0.85841286, 0.02831742,
          0.02831742

In [5]:
lk_atac = interface._model.likelihood(lr_t, "atac")

NameError: name 'interface' is not defined

In [43]:
from congas import log_sum_exp
lk_atac = lk_atac + torch.log(lr_t["mixture_weights_atac"]).reshape([2,1,1])

In [46]:
torch.exp(lk_atac - log_sum_exp(lk_atac))

tensor([[[9.9939e-01, 9.9989e-01, 1.4069e-04,  ..., 6.5166e-03,
          5.3351e-01, 1.5664e-07],
         [9.3591e-01, 1.6383e-01, 1.8467e-02,  ..., 6.9422e-01,
          8.2800e-01, 1.4531e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         ...,
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01]],

        [[6.1418e-04, 1.1402e-04, 9.9986e-01,  ..., 9.9348e-01,
          4.6649e-01, 1.0000e+00],
         [6.4089e-02, 8.3617e-01, 9.8153e-01,  ..., 3.0578e-01,
          1.7200e-01, 8.5469e-01],
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.9646e-01, 4.9646e-01],
         ...,
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.964

TypeError: exp(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

dict_keys(['mixture_weights_rna', 'mixture_weights_atac', 'NB_size_rna', 'segment_factor_rna', 'NB_size_atac', 'segment_factor_atac', 'CNV_probabilities', 'CNA', 'assignment_probs_rna', 'assignment_rna', 'assignment_probs_atac', 'assignment_atac'])